## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [38]:
# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the data (replace "None" with your own code)
fp = r'shopping_centers.txt'
data = pd.read_csv(fp, sep=';')

In [39]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

     id       name                                           addr
0  1000       Itis             Itäkatu 1, 00930 Helsinki, Finland
1  1001      Forum    Mannerheimintie 20, 00100 Helsinki, Finland
2  1002  Iso_omena          Piispansilta 11, 02230 Espoo, Finland
3  1003      Sello        Leppävaarankatu 3, 02600 Espoo, Finland
4  1004      Jumbo     Vantaanportinkatu 3, 01510 Vantaa, Finland
5  1005       REDI  Hermannin rantatie 5, 00580 Helsinki, Finland
6  1006     Tripla                        00240 Helsinki, Finland


- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [40]:
# import modules
from geopandas.tools import geocode

# Geocode the addresses using Nominatim
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_VU', timeout=4)

,geometry,address
0,POINT (25.08294 60.21170),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (24.93858 60.16893),"Apollo Street Bar, 20, Mannerheimintie, Keskus..."
2,POINT (24.73779 60.16294),"Sports Academy, 11, Piispansilta, Nuottaniemi,..."
3,POINT (24.81433 60.21770),"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete..."
4,POINT (24.96282 60.29245),"Stockmann, 3, Vantaanportinkatu, Pakkala, Avia..."
5,POINT (24.97904 60.18702),"Silta, 5, Hermannin rantatie, Kalasatama, Sörn..."
6,POINT (24.92338 60.20076),"Pasila, Helsinki, Helsingin seutukunta, Uusima..."


In [41]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                    geometry  \
0  POINT (25.08294 60.21170)   
1  POINT (24.93858 60.16893)   
2  POINT (24.73779 60.16294)   
3  POINT (24.81433 60.21770)   
4  POINT (24.96282 60.29245)   
5  POINT (24.97904 60.18702)   
6  POINT (24.92338 60.20076)   

                                             address  
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  
1  Apollo Street Bar, 20, Mannerheimintie, Keskus...  
2  Sports Academy, 11, Piispansilta, Nuottaniemi,...  
3  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...  
4  Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...  
5  Silta, 5, Hermannin rantatie, Kalasatama, Sörn...  
6  Pasila, Helsinki, Helsingin seutukunta, Uusima...  


In [42]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [45]:
# existing data.crs (EPSG:4326), which is based on WGS84. We need to reproject it to EPSG:3879, which is cartesian

# import required library
from pyproj import CRS

# perform data projection
geo = geo.to_crs(CRS.from_epsg(3879))


In [46]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

epsg:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [47]:
# Join the tables
geodata = geo.join(data)


,geometry,address,id,name,addr
0,POINT (25504598.602 6677662.109),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1000,Itis,"Itäkatu 1, 00930 Helsinki, Finland"
1,POINT (25496590.247 6672895.892),"Apollo Street Bar, 20, Mannerheimintie, Keskus...",1001,Forum,"Mannerheimintie 20, 00100 Helsinki, Finland"
2,POINT (25485440.532 6672255.563),"Sports Academy, 11, Piispansilta, Nuottaniemi,...",1002,Iso_omena,"Piispansilta 11, 02230 Espoo, Finland"
3,POINT (25489707.583 6678342.162),"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...",1003,Sello,"Leppävaarankatu 3, 02600 Espoo, Finland"
4,POINT (25497943.932 6686656.982),"Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...",1004,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland"


In [48]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496590.247 6672895.892)   
2  POINT (25485440.532 6672255.563)   
3  POINT (25489707.583 6678342.162)   
4  POINT (25497943.932 6686656.982)   

                                             address    id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  1000       Itis   
1  Apollo Street Bar, 20, Mannerheimintie, Keskus...  1001      Forum   
2  Sports Academy, 11, Piispansilta, Nuottaniemi,...  1002  Iso_omena   
3  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...  1003      Sello   
4  Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...  1004      Jumbo   

                                          addr  
0           Itäkatu 1, 00930 Helsinki, Finland  
1  Mannerheimintie 20, 00100 Helsinki, Finland  
2        Piispansilta 11, 02230 Espoo, Finland  
3      Leppävaarankatu 3, 02600 Espoo, Finland  
4   Vantaanportinkatu 3, 01510 Vantaa, Finland  


- Save the output as a Shapefile called `shopping_centers.shp` 

In [49]:
# Define output filepath
out_fp = r'shopping_centers.shp'

# Save file
geodata.to_file(out_fp)


In [50]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Geocoded output is stored in this file: shopping_centers.shp


## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [52]:
# create a ne col
geodata['buffer'] = None

- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [53]:
# units of geodata.crs are meters, so use 1500m as a buffer distance
geodata['buffer'] = geodata.buffer(1500)
geodata.head()

,geometry,address,id,name,addr,buffer
0,POINT (25504598.602 6677662.109),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1000,Itis,"Itäkatu 1, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,POINT (25496590.247 6672895.892),"Apollo Street Bar, 20, Mannerheimintie, Keskus...",1001,Forum,"Mannerheimintie 20, 00100 Helsinki, Finland","POLYGON ((25498090.247 6672895.892, 25498083.0..."
2,POINT (25485440.532 6672255.563),"Sports Academy, 11, Piispansilta, Nuottaniemi,...",1002,Iso_omena,"Piispansilta 11, 02230 Espoo, Finland","POLYGON ((25486940.532 6672255.563, 25486933.3..."
3,POINT (25489707.583 6678342.162),"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...",1003,Sello,"Leppävaarankatu 3, 02600 Espoo, Finland","POLYGON ((25491207.583 6678342.162, 25491200.3..."
4,POINT (25497943.932 6686656.982),"Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...",1004,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland","POLYGON ((25499443.932 6686656.982, 25499436.7..."


In [54]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496590.247 6672895.892)   
2  POINT (25485440.532 6672255.563)   
3  POINT (25489707.583 6678342.162)   
4  POINT (25497943.932 6686656.982)   

                                             address    id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  1000       Itis   
1  Apollo Street Bar, 20, Mannerheimintie, Keskus...  1001      Forum   
2  Sports Academy, 11, Piispansilta, Nuottaniemi,...  1002  Iso_omena   
3  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...  1003      Sello   
4  Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...  1004      Jumbo   

                                          addr  \
0           Itäkatu 1, 00930 Helsinki, Finland   
1  Mannerheimintie 20, 00100 Helsinki, Finland   
2        Piispansilta 11, 02230 Espoo, Finland   
3      Leppävaarankatu 3, 02600 Espoo, Finland   
4   Vantaanportinkatu 3, 01510 Vantaa, Finland   

                         

In [55]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

<class 'shapely.geometry.polygon.Polygon'>


In [56]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
5    7.0
6    7.0
dtype: float64


- Replace the values in `geometry` column with the values of `buffer` column:

In [59]:
geodata['geometry'] = geodata['buffer']

In [60]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                                            geometry  \
0  POLYGON ((25506098.602 6677662.109, 25506091.3...   
1  POLYGON ((25498090.247 6672895.892, 25498083.0...   
2  POLYGON ((25486940.532 6672255.563, 25486933.3...   
3  POLYGON ((25491207.583 6678342.162, 25491200.3...   
4  POLYGON ((25499443.932 6686656.982, 25499436.7...   

                                             address    id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  1000       Itis   
1  Apollo Street Bar, 20, Mannerheimintie, Keskus...  1001      Forum   
2  Sports Academy, 11, Piispansilta, Nuottaniemi,...  1002  Iso_omena   
3  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...  1003      Sello   
4  Stockmann, 3, Vantaanportinkatu, Pakkala, Avia...  1004      Jumbo   

                                          addr  \
0           Itäkatu 1, 00930 Helsinki, Finland   
1  Mannerheimintie 20, 00100 Helsinki, Finland   
2        Piispansilta 11, 02230 Espoo, Finland   
3      Leppävaarankatu 3

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [61]:
# keep only required cols
cols = ['id', 'name', 'geometry']

geodata = geodata[cols]
geodata.head(3)

,id,name,geometry
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1001,Forum,"POLYGON ((25498090.247 6672895.892, 25498083.0..."
2,1002,Iso_omena,"POLYGON ((25486940.532 6672255.563, 25486933.3..."


## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `25858 people live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [72]:
# Read population grid data for 2018 into a variable `pop`. 

# load modules
from pyproj import CRS
import requests
import geojson

#1. Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

#2. keep only the required cols
pop = pop[['geometry', 'asukkaita']]

# Change the name of a column
pop = pop.rename(columns={'asukkaita': 'pop18'})

# assign projection to pop
pop.crs = CRS.from_epsg(3879)

# confirm projections for both layers: pop and geodata
# print('Does pop and geodata layers have the same crs? - ', pop.crs == geodata.crs)

#3. perform spatial join
join = gpd.sjoin(geodata, pop, how='inner', op='contains')

#4. group data by name of mall and then sum population col
grouped = join.groupby('name')['pop18'].sum()
grouped

name
Forum        55825
Iso_omena    26694
Itis         19930
Jumbo        10317
REDI         24540
Sello        21490
Tripla       18791
Name: pop18, dtype: int64

In [73]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 3167
                                            geometry  pop18
0  MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...    108
1  MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...    273
2  MULTIPOLYGON Z (((25476999.994 6675749.004 0.0...    239


In [74]:
# Create a spatial join between grid layer and buffer layer. 
# implemented above

In [76]:
# Report how many people live within 1.5 km distance from each shopping center
# implemented above
# name         Pop
# Forum        55825
# Iso_omena    26694
# Itis         19930
# Jumbo        10317
# REDI         24540
# Sello        21490
# Tripla       18791

**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Not bad!

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)